## Import necessary packages and disable RDKIT logging

In [ ]:
from rdkit import Chem
from itertools import product
from rdkit import RDLogger
import mols2grid
RDLogger.DisableLog('rdApp.*')

## Single point replacement

In [ ]:

# This is the core with '*' in replacement point
core_smiles = "CC1(C)CCC(CC1)NC[C@@H](O)-*"
core = Chem.MolFromSmiles(core_smiles)

# Defining '*' as smarts
rgroup_smarts = Chem.MolFromSmarts("[#0]")


# smiles_frag indicated python list of smiles. You can open the create a list as below
'''
with open('fragment_file_path', 'r') as file:
    smiles_frag = file.readlines()
'''
rgroup_smiles = smiles_frag
rgroup_mols = [Chem.MolFromSmiles(smi) for smi in rgroup_smiles]

# Create a empty list products where smiles will be appended after replacement. 
# You can make it as generator that yields one smiles at a time.
products = []

for r_mol in rgroup_mols:
    if r_mol is None:
        continue

    replaced = Chem.ReplaceSubstructs(
        core,
        rgroup_smarts,
        r_mol,
        replaceAll=True
    )

    if replaced:
        try:
            prod = replaced[0]
            Chem.SanitizeMol(prod)
            products.append(Chem.MolToSmiles(prod))
        except:
            pass


# Use mols2grid to display the replaced structure
products = sorted(set(products))
smi = [Chem.MolFromSmiles(i) for i in products]
mols2grid.display(smi)

## Multi point replacement

In [ ]:
# same as single point replacement but for can be used to replace multiple positions at same time

core = Chem.MolFromSmiles("O[C@@H](*)CN-*")

rgroup_smarts = Chem.MolFromSmarts("[#0]")

# smiles_frag can be same fragment list or different list for two different position
rgroup_set_1 = smiles_frag
rgroup_set_2 = smiles_frag

rgroup_mols_1 = [Chem.MolFromSmiles(s) for s in rgroup_set_1]
rgroup_mols_2 = [Chem.MolFromSmiles(s) for s in rgroup_set_2]

final_products = []

# I used itertools.product() module which produces every combination of two list

'''

Example itertools.product()

from itertools import product
print(list(product([1, 2], [3, 4])))

output: [(1, 3), (1, 4), (2, 3), (2, 4)]

'''
from itertools import product

for r1, r2 in product(rgroup_mols_1, rgroup_mols_2):
    mol_step1 = Chem.ReplaceSubstructs(core, rgroup_smarts, r1, replaceAll=False)[0]
    mol_step2 = Chem.ReplaceSubstructs(mol_step1, rgroup_smarts, r2, replaceAll=False)[0]
    if '.' in Chem.MolToSmiles(mol_step2):
        pass
    else:
        try:
            Chem.SanitizeMol(mol_step2)
            final_products.append(Chem.MolToSmiles(mol_step2))
        except:
            pass

products = sorted(set(final_products))
smi = [Chem.MolFromSmiles(i) for i in products]
mols2grid.display(smi)